<a href="https://colab.research.google.com/github/michelanthony/colab/blob/main/Inference_SatGas_FourierNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch==1.11.0 torchvision torchaudio torchtext

In [ ]:
!git clone https://github.com/gegewen/ufno.git


In [ ]:
%cd /content/ufno

In [ ]:
import torch
import numpy as np
from ufno import *
from lploss import *
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda:0')
DATA_DIR='/content/drive/MyDrive'

In [ ]:
test_u = torch.load(f'{DATA_DIR}/sg_test_u.pt')
test_u = test_u[:200]
print(test_u.shape)

test_a = torch.load(f'{DATA_DIR}/sg_test_a.pt')
test_a = test_a[:200]
print(test_a.shape)

In [ ]:
#train_u = torch.load(f'{DATA_DIR}/sg_train_u.pt')
#print(train_u.shape)

#train_a = torch.load(f'{DATA_DIR}/sg_train_a.pt')
#print(train_a.shape)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)


In [ ]:
mode1 = 10
mode2 = 10
mode3 = 10
width = 36
device = torch.device('cuda:0')
model = Net3d(mode1, mode2, mode3, width)
    
model.to(device)

In [ ]:
# prepare for calculating x direction derivatives 
time_grid = np.cumsum(np.power(1.421245, range(24)))
time_grid /= np.max(time_grid)
grid_x = test_a[0,0,:,0,-3]
grid_dx = grid_x[1:-1] + grid_x[:-2]/2 + grid_x[2:]/2
grid_dx = grid_dx[None, None, :, None].to(device)

In [ ]:
epochs = 50
e_start = 0
learning_rate = 0.001
scheduler_step = 2
scheduler_gamma = 0.9
learning_rate

In [ ]:
batch_size = 2

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=True)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
myloss = LpLoss(size_average=False)

In [ ]:
# train_l2 = 0.0
#for ep in range(1,epochs+1):
#    model.train()
#    train_l2 = 0
#    counter = 0
#    for x, y in train_loader:
#        x, y = x.to(device), y.to(device)
#        dy = (y[:,:,2:,:] - y[:,:,:-2,:])/grid_dx        
#        optimizer.zero_grad()
#        
#        mask = (x[:,:,:,0:1,0]!=0).repeat(1,1,1,24)
#        dy = (y[:,:,2:,:] - y[:,:,:-2,:])/grid_dx
#        pred = model(x).view(-1,96,200,24)
#        dy_pred = (pred[:,:,2:,:] - pred[:,:,:-2,:])/grid_dx
#        ori_loss = 0
#        der_loss = 0
#        
#        # original loss
#        for i in range(batch_size):
#            ori_loss += myloss(pred[i,...][mask[i,...]].reshape(1, -1), y[i,...][mask[i,...]].reshape(1, -1))
#
#        # 1st derivative loss
#        dy_pred = (pred[:,:,2:,:] - pred[:,:,:-2,:])/grid_dx
#        mask_dy = mask[:,:,:198,:]
#        for i in range(batch_size):
#            der_loss += myloss(dy_pred[i,...][mask_dy[i,...]].reshape(1, -1), dy[i,...][mask_dy[i,...]].view(1, -1))
#
#        loss = ori_loss + 0.5 * der_loss
#        
#        loss.backward()
#        optimizer.step()
#        train_l2 += loss.item()
#
#        counter += 1
#        if counter % 10 == 0:
#            print(f'epoch: {ep}, batch: {counter}/{len(train_loader)}, train loss: {loss.item()/batch_size:.4f}')
#        
#    scheduler.step()
#    
#    print(f'epoch: {ep}, train loss: {train_l2/test_a.shape[0]:.4f}')
#    
#    lr_ = optimizer.param_groups[0]['lr']
#    if ep % 2 == 0:
#        PATH = f'/content/drive/MyDrive/UFNO/T4_SG_UFNO_{ep}ep_{width}width_{mode1}m1_{mode2}m2_{test_a.shape[0]}train_{lr_:.2e}lr'
#        torch.save(model, PATH)

In [ ]:
# Save Model

In [ ]:
#model = torch.load('/content/drive/MyDrive/UFNO/T4_SG_UFNO_28ep_36width_10m1_10m2_200train_2.29e-04lr')

#torch.save(model, '/content/drive/MyDrive/UFNO/T4_model_last.pt')

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/UFNO/T4_model_state_dict_last.pt')

In [ ]:
#============================================
# Reload Model
#============================================


In [ ]:
model2 = torch.load('/content/drive/MyDrive/UFNO/T4_model_last.pt')
#model2 = torch.load('/content/drive/MyDrive/UFNO/T4_SG_UFNO_28ep_36width_10m1_10m2_200train_2.29e-04lr')
model2.eval()

In [ ]:
x = test_a[1:2].to(device)
y = test_u[1:2].to(device)

print(x.shape)
print(y.shape)

In [ ]:
ypred = model2(x)
print(ypred.shape)

In [ ]:
# Prepare for plotting
dx = np.cumsum(3.5938*np.power(1.035012, range(200))) + 0.1
X, Y = np.meshgrid(dx, np.linspace(0,200,num = 96))

def pcolor(x):
    plt.jet()
    return plt.pcolor(X[:thickness, :], Y[:thickness, :], np.flipud(x), shading='auto')

times = np.cumsum(np.power(1.421245, range(24)))
time_print = []
for t in range(times.shape[0]):
    if times[t]<365:
        title = str(int(times[t]))+' d'
        time_print = np.append(time_print,title)
    else:
        title = f'{round(int(times[t])/365, 1)} y' 
        time_print = np.append(time_print,title)

dnorm_inj = lambda a : (a * (3e6 - 3e5) + 3e5) / (1e6 /365*1000/1.862)
dnorm_temp = lambda a : a * (180 - 30) + 30
dnorm_P = lambda a : a * (300 - 100) + 100
dnorm_lam = lambda a : a * 0.4 + 0.3
dnorm_Swi = lambda a : a * 0.2 + 0.1


In [ ]:
# Choix du test de validation : 

i=51
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a[i:i+1,...], 
                                                                         test_u[i:i+1,...]), 
                                          batch_size=1, 
                                          shuffle=False)

In [ ]:
x, y = next(iter(test_loader))
x, y = x.to(device), y.to(device)
pred = model2(x).view(-1,96,200,24)

y_plot = y.cpu().detach().numpy()
x_plot = x.cpu().detach().numpy()
pred_plot = pred.cpu().detach().numpy()

# reservior have different thickness as marked in the permeability map
mask = x_plot[0,:,:,0,0] != 0 
thickness = sum(mask[:,0])

# extract input parameters
poro_map = x_plot[0,:,:,0,2][mask].reshape((thickness, -1))
kr_map = np.exp(x_plot[0,:,:,0,0][mask].reshape((thickness, -1))*15)
kz_map = np.exp(x_plot[0,:,:,0,1][mask].reshape((thickness, -1))*15)

inj_rate = dnorm_inj(x_plot[0,0,0,0,4])
temperature = dnorm_temp(x_plot[0,0,0,0,5])
pressure = dnorm_P(x_plot[0,0,0,0,6])
Swi = dnorm_Swi(x_plot[0,0,0,0,7])
lam = dnorm_lam(x_plot[0,0,0,0,8])

print(f'Params: injection rate: {inj_rate:.2f} MT/yr, temperature: {temperature:.1f} C, initial pressure: {pressure:.1f} bar, Swi: {Swi:.2f}, lan: {lam:.2f}')


In [ ]:
t_lst = [14,20,23]
plt.figure(figsize=(15,6))    
for j, t in enumerate(t_lst):   
    plt.subplot(4,3,j+1)
    if j == 2:
        pcolor(poro_map)
        plt.title('$\phi$ (-)')
    elif j == 1:
        pcolor(kz_map)
        plt.title('$k_z$ (mD)')
    else:
        pcolor(kr_map)
        plt.title('$k_r$ (mD)')
    plt.colorbar(fraction=0.02)
    plt.xlim([0,3500])        

    plt.subplot(4,3,j+4)
    pcolor(y_plot[0,:,:,t][mask].reshape((thickness, -1)))
    plt.title('$SG$ (-), '+f't={time_print[t]}')
    plt.colorbar(fraction=0.02)
    plt.xlim([0,3500])

    plt.subplot(4,3,j+7)
    pcolor(pred_plot[0,:,:,t][mask].reshape((thickness, -1)))
    plt.title('$\hat{SG}$ (-), '+f't={time_print[t]}')
    plt.colorbar(fraction=0.02)
    plt.xlim([0,3500])

    plt.subplot(4,3,j+10)
    pcolor((pred_plot[0,:,:,t][mask].reshape((thickness, -1))-y_plot[0,:,:,t][mask].reshape((thickness, -1))))
    plt.colorbar(fraction=0.02)
    plt.title('|$SG-\hat{SG}$|, '+f't={time_print[t]}')
    plt.xlim([0,3500])
plt.tight_layout()
plt.show()